<a href="https://colab.research.google.com/github/kwanhong66/TodayILearned/blob/master/python_ml_complete_guide/2%EC%9E%A5/2_4_Model_Selection_%EB%AA%A8%EB%93%88_%EC%86%8C%EA%B0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Book] 파이썬 머신러닝 완벽 가이드
- 예제 코드 transcription
- 개념, 프로세스, API, Best Practice 등 익히기
- 책 예제 코드 github (https://github.com/wikibook/pymldg-rev)

## 2.4 Model Selection 모듈 소개
- sklearn.model_selection 모듈
- 학습 데이터와 테스트 데이터 셋 분리: train_test_split
- 교차 검증(cross validation) 분할 및 평가: KFold, StratifiedKFold, cross_val_score
- Estimator의 하이퍼 파라미터 튜닝: GridSearchCV

### 학습/테스트 데이터 세트 분리 - train_test_split()

In [2]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

iris = load_iris()
dt_clf = DecisionTreeClassifier()
train_data = iris.data
train_label = iris.target
dt_clf.fit(train_data, train_label)

# 학습 데이터 셋으로 예측 수행
pred = dt_clf.predict(train_data)
print('예측 정확도:', accuracy_score(train_label, pred))

예측 정확도: 1.0


- 이미 학습한 데이터 셋으로 예측하였기 때문에, 학습 시 보았던 데이터는 정답을 알고 있음
- 예측을 수행하는 데이터 셋은 학습에 사용한 학습용 데이터 셋이 아닌, 별도의 전용 테스트 데이터 셋이어야 함
- train_test_split()으로 학습 및 테스트 데이터 셋을 쉽게 분리

In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

dt_clf = DecisionTreeClassifier()
iris_data = load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, \
                                                    test_size=0.3, random_state=121)

- train_test_split()을 이용해 테스트 데이터 셋을 전체의 30%로, 학습 데이터 셋을 70%로 분리
- random_state=121로 설정

In [4]:
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))

예측 정확도: 0.9556


### 교차 검증
- 고정된 학습 데이터와 테스트 데이터로 평가를 하면, 모델이 테스트 데이터에만 최적의 성능을 보이게 편향되어 학습이 될 수 있음
- 고정된 테스트 데이터 기준으로 평가를 하여, 해당 성능을 기준으로 모델을 개선하면 보지 못한 다른 테스트 데이터가 들어올 경우, 성능이 저하되는 현상이 나타날 수 있음 (일반화 성능 저하)
- 이러한 문제점을 개선하기 위해, 교차 검증으로 다양한 학습과 평가를 수행

#### K 폴드 교차 검증 (K-fold cross validation)

- K개의 데이터 fold 셋을 만들어, k번만큼 각 fold 셋에 학습과 검증 평가를 반복적으로 수행

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

iris = load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state=156)

# 5개의 fold 셋으로 분리하는 KFold 객체와 fold 셋 별 정확도를 담을 리스트 객체 생성
kfold = KFold(n_splits=5)
cv_accuracy = []
print('붓꽃 데이터 세트 크기:', features.shape[0])

붓꽃 데이터 세트 크기: 150


In [7]:
n_iter = 0

# KFold 객체의 split()을 호출하면, fold 별 학습용, 검증용 테스트의 row index를 array로 반환
for train_index, test_index, in kfold.split(features):
  # kfold.split()으로 반환된 인덱스를 이용해 학습용, 검증용 테스트 데이터의 추출
  X_train, X_test = features[train_index], features[test_index]
  y_train, y_test = label[train_index], label[test_index]
  # 학습 및 예측
  dt_clf.fit(X_train, y_train)
  pred = dt_clf.predict(X_test)
  n_iter += 1
  # 반복 시마다 정확도 측정
  accuracy = np.around(accuracy_score(y_test, pred), 4)
  train_size = X_train.shape[0]
  test_size = y_train.shape[0]
  print('#{} 교차 검증 정확도 : {}, 학습 데이터 크기: {}, 검증 데이터 크기: {}'
        .format(n_iter, accuracy, train_size, test_size))
  print('#{} 검증 세트 인덱스: {}'.format(n_iter, test_index))
  cv_accuracy.append(accuracy)

# 개별 iteration 별, 정확도를 합하여 평균 정확도 계산
print('## 평균 검증 정확도:', np.mean(cv_accuracy))

#1 교차 검증 정확도 : 1.0, 학습 데이터 크기: 120, 검증 데이터 크기: 120
#1 검증 세트 인덱스: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
#2 교차 검증 정확도 : 0.9667, 학습 데이터 크기: 120, 검증 데이터 크기: 120
#2 검증 세트 인덱스: [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]
#3 교차 검증 정확도 : 0.8667, 학습 데이터 크기: 120, 검증 데이터 크기: 120
#3 검증 세트 인덱스: [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]
#4 교차 검증 정확도 : 0.9333, 학습 데이터 크기: 120, 검증 데이터 크기: 120
#4 검증 세트 인덱스: [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
#5 교차 검증 정확도 : 0.7333, 학습 데이터 크기: 120, 검증 데이터 크기: 120
#5 검증 세트 인덱스: [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]
## 평균 검증 정확도: 0.9


- 5번의 교차 검증 결과 평균 검증 정확도는 0.9
- 교차 검증 시마다 검증 세트의 인덱스가 달라짐 확인 가능

#### Stratified K-fold 
- Stratified K-fold은 불균형한(imbalanced) 비율의 label 데이터 집합을 위한 k-fold 방식
- 원본 데이터의 label 분포를 고려하여, 비율을 유지하여 학습과 검증 데이터 셋을 분배

In [9]:
import pandas as pd

iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label'] = iris.target
iris_df['label'].value_counts()

2    50
1    50
0    50
Name: label, dtype: int64

- 붓꽃 데이터의 label(품종) 비율은 모두 50개로 동일
- KFold에서 이슈가 발생하는 현상 확인

In [11]:
kfold = KFold(n_splits=3)
n_iter = 0
for train_index, test_index in kfold.split(iris_df):
  n_iter += 1
  label_train = iris_df['label'].iloc[train_index]
  label_test = iris_df['label'].iloc[test_index]
  print('## 교차 검증: {}'.format(n_iter))
  print('학습 레이블 데이터 분포:\n', label_train.value_counts())
  print('검증 레이블 데이터 분포:\n', label_test.value_counts())

## 교차 검증: 1
학습 레이블 데이터 분포:
 2    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    50
Name: label, dtype: int64
## 교차 검증: 2
학습 레이블 데이터 분포:
 2    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    50
Name: label, dtype: int64
## 교차 검증: 3
학습 레이블 데이터 분포:
 1    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    50
Name: label, dtype: int64


- 교차 검증 시마다 3개의 fold 셋으로 만들어지는 학습 label과 검증 label이 다른 값이 추출
- 1, 2 label로 학습한 모델은 0 label만 가진 검증 데이터 셋을 예측할 수 없음
- StratifiedKFold는 이렇게 KFold로 분할된 label 데이터 셋이 전체 label 값의 분포를 반영하지 못 하는 문제를 해결

In [14]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
n_iter = 0

for train_index, test_index in skf.split(iris_df, iris_df['label']):
  n_iter = 1
  label_train = iris_df['label'].iloc[train_index]
  label_test = iris_df['label'].iloc[test_index]
  print('## 교차 검증: {}'.format(n_iter))
  print('학습 레이블 데이터 분포:\n', label_train.value_counts())
  print('검증 레이블 데이터 분포:\n', label_test.value_counts())

## 교차 검증: 1
학습 레이블 데이터 분포:
 2    34
1    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    17
0    17
2    16
Name: label, dtype: int64
## 교차 검증: 1
학습 레이블 데이터 분포:
 1    34
2    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    17
0    17
1    16
Name: label, dtype: int64
## 교차 검증: 1
학습 레이블 데이터 분포:
 0    34
2    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    17
1    17
0    16
Name: label, dtype: int64


- 학습 label과 검증 label의 데이터 값 분포가 동일하게 할당
- 전체 label에 대해 학습하고, 검증 가능

In [16]:
dt_clf = DecisionTreeClassifier(random_state=156)

skfold = StratifiedKFold(n_splits=3)
n_iter = 0
cv_accuracy = []

# StratifiedKFold의 split() 호출시 반드시 레이블 데이터 세트도 추가 입력 필요
for train_index, test_index in skfold.split(features, label):
  # split()으로 반환된 인덱스를 이용해 학습용, 검증용 테스트 데이터 추출
  X_train, X_test = features[train_index], features[test_index]
  y_train, y_test = label[train_index], label[test_index]
  # 학습 및 예측
  dt_clf.fit(X_train, y_train)
  pred = dt_clf.predict(X_test)

  # 반복 시마다 정확도 측정
  n_iter += 1
  accuracy = np.round(accuracy_score(y_test, pred), 4)
  train_size = X_train.shape[0]
  test_size = X_test.shape[0]
  print('#{} 교차 검증 정확도 :{}, 학습 데이터 크기: {}, 검증 데이터 크기: {}'
        .format(n_iter, accuracy, train_size, test_size))
  print('#{} 검증 세트 인덱스: {}'.format(n_iter, test_index))
  cv_accuracy.append(accuracy)

# 교차 검증별 정확도 및 평균 정확도 계산
print('## 교차 검증별 정확도:', np.round(cv_accuracy, 4))
print('## 평균 검증 정확도:', np.mean(cv_accuracy))

#1 교차 검증 정확도 :0.98, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#1 검증 세트 인덱스: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]
#2 교차 검증 정확도 :0.94, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#2 검증 세트 인덱스: [ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]
#3 교차 검증 정확도 :0.98, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#3 검증 세트 인덱스: [ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]
## 교차 검증별 정확도: [0.98 0.94 0.98]
## 평균 검증 정확도: 0.9666666666666667


- 3개의 Stratified K-fold로 교차 검증하여 평균 검증 정확도 약 96.67%
- Stratified K-fold는 원본 데이터의 label 분포 특성을 반영하여, 학습 및 검증 데이터 세트를 만듬

#### 교차 검증을 보다 간편하게 - cross_val_score()
- 교차 검증을 좀 더 편리하게 수행할 수 있게 해주는 API
  * 1) fold 셋을 설정
  * 2) 학습 및 검증 데이터 셋의 index 추출하는 iteration
  * 3) 반복적으로 학습과 예측 수행하여, 예측 성능 반환
- 이러한 일련의 과정을 한번에 수행해주는 API
- cross_val_score() 수행 후 반환 값은 scoring 파라미터로 지정된 성능 지표 측정값을 배열 형태로 반환

In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris_data.data
label = iris_data.target

# 성능 지표는 정확도(accuracy), 교차 검증 세트는 3개
scores = cross_val_score(dt_clf, data, label, scoring='accuracy', cv=3)
print('교차 검증별 정확도:', np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

교차 검증별 정확도: [0.98 0.94 0.98]
평균 검증 정확도: 0.9667


### GridSearchCV - 교차 검증과 최적 하이퍼 파라미터 튜닝을 한 번에
- Classifier나 Regressor와 같은 알고리즘에 사용하는 하이퍼 파라미터를 순차적으로 입력하면서 편리하게 최적의 파라미터를 도출
- 교차 검증 기반으로 최적의 하이퍼 파라미터 값 찾음
  * 데이터 셋을 cross validation을 위한 학습/테스트 셋으로 자동 분할
  * parameter grid에 기술된 모든 파라미터를 순차적으로 적용해 최적의 파라미터 찾음

In [0]:
grid_parameters = {'max_depth': [1, 2, 3],
                   'min_samples_split': [2, 3]}

In [0]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# 데이터를 로딩하고 학습 데이터와 테스트 데이터 분리
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target,
                                                    test_size=0.2, random_state=121)
dtree = DecisionTreeClassifier()

### 파라미터를 딕셔너리 형태로 설정
parameters = {'max_depth': [1, 2, 3], 'min_samples_split': [2, 3]}

- train_test_split()으로 학습 데이터와 테스트 데이터 분리
- 학습 데이터에서 GridSearchCV를 이용해 최적의 하이퍼 파라미터 추출

In [22]:
import pandas as pd

# param_grid의 하이퍼 파라미터를 3개의 train, test set fold로 나누어 테스트 수행 설정
### refit=True가 default. True이면 가장 좋은 파라미터 설정으로 재학습
grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True)

# 붓꽃 학습 데이터로 param_grid의 하이퍼 파라미터를 순차적으로 학습/평가
grid_dtree.fit(X_train, y_train)

# GridSearchCV 결과를 추출해 DataFrame으로 변환
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score',
           'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


- 하이퍼 파라미터를 순차적으로 변경하면서 학습 및 평가 수행
- params: 학습 및 평가를 수행할 때마다 적용된 개별 하이퍼 파라미터 값
- rank_test_score: 하이퍼 파라미터 별로 성능이 좋은 score 순위
- mean_test_score: 개별 하이퍼 파라미터 별로 CV의 fold 테스트 세트에 대해 총 수행한 평가 평균값

In [23]:
print('GridSearchCV 최적 파라미터:', grid_dtree.best_params_)
print('GridSearchCV 최고 정확도:{0:.4f}'.format(grid_dtree.best_score_))

GridSearchCV 최적 파라미터: {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도:0.9750


In [24]:
# GridSearchCV의 refit으로 이미 학습된 estimator 반환
estimator = grid_dtree.best_estimator_

# GridSearchCV의 best_estimator_는 이미 최적 학습이 됐으므로 별도 학습 불필요
pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))

테스트 데이터 세트 정확도: 0.9667
